# Langchain

Placer les fichier ```new_google_tool.py``` et ```new_google_wrapper.py``` dans le même dossier que ce script

Pour le moment les classes n'ont été que très légèrements modifiées.

Dans le futur on pourrait y mettre le code de la cellule suivante SerpAPI

Changer la clé d'API


In [1]:
import os
from new_google_tool import MyGoogleTrendsQueryRun as GoogleTrendsQueryRun
from new_google_wrapper import MyGoogleTrendsAPIWrapper as GoogleTrendsAPIWrapper

os.environ["SERPAPI_API_KEY"] = "cf7a3d5a2dd56565ef56451a80d9ea4a5540690bc11c12fc84417be5b4e17f27"
tool = GoogleTrendsQueryRun(api_wrapper=GoogleTrendsAPIWrapper())

print(tool.run({"query":"vomissements", "geo":"FR", "date":"2018-01-01 2018-09-27", "tz":"2", "data_type":"TIMESERIES"}))

Query: vomissements
Date From: Jan 1,, 2018
Min Value: 0
Max Value: 100
Average Value: 41.837037037037035
Percent Change: -37.93103448275862%
Trend values: 87, 100, 91, 63, 82, 51, 62, 68, 46, 0, 58, 57, 95, 86, 58, 63, 62, 75, 40, 48, 57, 67, 55, 71, 86, 56, 80, 95, 54, 0, 75, 78, 0, 68, 63, 0, 50, 43, 54, 0, 46, 0, 78, 85, 48, 75, 72, 69, 94, 49, 66, 54, 45, 0, 53, 49, 53, 96, 49, 0, 55, 82, 98, 46, 58, 39, 67, 77, 94, 55, 51, 0, 41, 61, 0, 49, 0, 88, 42, 56, 43, 53, 46, 61, 45, 51, 37, 76, 44, 0, 75, 82, 62, 39, 0, 69, 62, 69, 0, 100, 0, 62, 52, 0, 73, 64, 57, 0, 56, 0, 0, 47, 50, 0, 48, 0, 55, 44, 44, 48, 0, 41, 66, 76, 49, 48, 42, 0, 63, 90, 0, 52, 72, 49, 38, 43, 41, 57, 0, 0, 0, 0, 63, 0, 0, 53, 0, 43, 58, 0, 58, 0, 77, 0, 0, 51, 0, 0, 51, 65, 66, 52, 57, 41, 0, 44, 62, 0, 0, 49, 39, 0, 56, 0, 0, 57, 0, 0, 54, 40, 0, 0, 41, 52, 44, 0, 0, 83, 0, 0, 0, 39, 65, 56, 0, 0, 0, 0, 43, 46, 68, 0, 71, 51, 50, 0, 49, 68, 0, 55, 48, 42, 0, 41, 65, 56, 62, 46, 0, 42, 51, 52, 53, 81, 0, 44, 

# SerpAPI

Nombre de requête limité à 100 par mois

Changer la clé API et adatper les autres paramètres selon le besoin

In [14]:
import pandas as pd
import datetime as dt
# logger.info("On charge les données de Google Trends depuis le fichier")
all_words_df_2024 = pd.read_csv('/home/maxime/Documents/WORKSPACES/ARS_Project/data/features/google_trends/trends_2019_2024.csv', parse_dates=['date_entree'])
all_words_df_2018 = pd.read_csv('/home/maxime/Documents/WORKSPACES/ARS_Project/data/features/google_trends/trends_2018.csv', parse_dates=['date_entree'])
all_words_df_2018.date_entree = all_words_df_2018.date_entree.dt.date
all_words_df_2024.date_entree = all_words_df_2024.date_entree.dt.date
all_words_df = pd.concat([all_words_df_2018.loc[all_words_df_2018.date_entree < dt.datetime(2019, 1, 1).date()], all_words_df_2024])
all_words_df.to_csv('/home/maxime/Documents/WORKSPACES/ARS_Project/data/features/google_trends/trends.csv', index=False)

In [ ]:
from serpapi import GoogleSearch
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt


BATCH_SIZE = 270
START = dt.datetime(2018, 1, 1)
STOP_DATA = dt.datetime(2024, 1, 1)
API_KEY = "2db567698593dfdbc27232e2244cd1346cac3c105e2b93f11334db10e8536bde"

bow = [
    "diarrhée", "vomissements", "toux", "éruption cutanée", "infection urinaire",
    "hopital", "médecin", "pharmacie", "médicament", "vaccin", "maladie", "fièvre",
    "grippe", "rhume", "angine", "otite", "allergie", "asthme", "diabète", "obésité",
    "tabac", "alcool", "drogue", "stress", "cancer", "dépression", "mal de tête",
    "douleur thoracique", "palpitations", "essoufflement", "vertiges", "crampes abdominales",
    "saignements", "douleur abdominale", "hypothermie", "hyperthermie", "appendicite",
    "méningite", "pneumonie", "maladie cardiaque", "AVC", "infection respiratoire",
    "gastro-entérite", "infection cutanée", "insuffisance cardiaque", "maladie rénale",
    "épilepsie", "migraines", "maladie de Crohn", "colite ulcéreuse", "accident de voiture",
    "fracture", "entorse", "brûlure", "empoisonnement", "chute", "noyade", "asphyxie",
    "crise de panique", "schizophrénie", "trouble bipolaire", "démence", "tentative de suicide",
    "urgences", "urgence médicale", "service d'urgence", "douleur", "S.O.S. médecin"
]

def date_range(start_date, stop_date, batch_size):
    current_date = start_date
    while current_date <= stop_date:
        yield current_date
        current_date += dt.timedelta(days=batch_size)

def get_all_dates_df(bow, i, start, end, batch_size):
    all_dates_df = pd.DataFrame()
    for date in date_range(start, end, batch_size):
        end_date = min(date + dt.timedelta(days=batch_size-1), end)

        group = ', '.join(bow[i:i+5])
        print(f"Récupération de {group} pour la période {date} - {end_date}")

        params = {
            "engine": "google_trends",
            "q": f"{group}",
            "geo": "FR",
            "date": f"{date.strftime('%Y-%m-%d')} {end_date.strftime('%Y-%m-%d')}",
            "tz": "0",
            "data_type": "TIMESERIES",
            "api_key": API_KEY
        }
        search = GoogleSearch(params)
        results = search.get_dict()
        try:
            interest_over_time = results["interest_over_time"]["timeline_data"]
            df = pd.DataFrame(interest_over_time)
            for query in df.iloc[0]["values"]:
                query = query["query"]
                df["trend_" + query] = df["values"].apply(lambda x: next(x[i]["value"] for i in range(len(x)) if x[i]["query"] == query))
                df["trend_" + query] = df["trend_" + query].apply(lambda x: x.replace('<', '').replace('>', ''))
                df["trend_" + query] = df["trend_" + query].astype(int)

            df.drop(columns=["values", "timestamp"], inplace=True)
            df["date_entree"] = pd.to_datetime(df["date"], format="%b %d, %Y")
            df.drop(columns=["date"], inplace=True)
            df.set_index("date_entree", inplace=True)
            all_dates_df = pd.concat([all_dates_df, df], axis=0)
        except:
            # Ce code permet de réessayer avec un batch plus petit mais cela ne résoud pas le problème, on décide donc de remplir de 0 plus tard
            # On perd 270 jours à chaque fois mais cette erreur ne se produit généralement que quand il n'y a pas (peu?) de données

            # if batch_size//2 > 1:
            #     print(f"On réessaye avec un batch plus petit de {batch_size//2} jours")
            #     try:
            #         all_dates_df = pd.concat([all_dates_df, get_all_dates_df(bow, i, date, end_date, batch_size//2)], axis=0)
            #     except:
            #         print(f"Problème avec {group} pour la période {date} - {end_date}")
            #         continue
            # else:
            #     print(f"Problème avec {group} pour la période {date} - {end_date}")
            continue
            

    return all_dates_df


# On récupère les tendance par lot de 5 mots max et 270 jours max
idx = pd.date_range(START, STOP_DATA, freq='D')
all_words_df = pd.DataFrame(index=idx)
for i in range(0, len(bow), 5):
    
    all_dates_df = get_all_dates_df(bow, i, START, STOP_DATA, BATCH_SIZE)

    # Fill missing dates
    idx = pd.date_range(START, STOP_DATA, freq='D')
    all_dates_df = all_dates_df.reset_index()

    all_dates_df.fillna(0, inplace=True)
    # all_dates_df.ffill(inplace=True)
    # all_dates_df.bfill(inplace=True)

    all_dates_df.set_index('date_entree', inplace=True)
    all_words_df = pd.concat([all_words_df, all_dates_df], axis=1)

all_words_df.reset_index(inplace=True)
all_words_df.rename({"index": "date_entree"}, axis=1, inplace=True)

# Tracer les graphiques
for i in range(0, len(bow), 5):
    fig, ax = plt.subplots(figsize=(20, 10))
    all_words_df.plot(x="date_entree", y=all_words_df.loc[:, all_words_df.columns != "date_entree"].columns.tolist()[i:i + 5], title="Google Trends in France", ax=ax)

# On sauvegarde le dataframe
all_words_df.to_csv('trends.csv', index=False)


# Pytrends

In [1]:
import pandas as pd                        
from pytrends.request import TrendReq
pytrend = TrendReq()

In [2]:
# Attention au ban de Google
pytrend.build_payload(kw_list=['pizza', 'bagel'], timeframe=['2022-09-01 2022-09-02', '2022-09-01 2022-09-02'])
pytrend.multirange_interest_over_time()

ResponseError: The request failed: Google returned a response with code 400

# APIfy

In [ ]:
from apify_client import ApifyClient

client = ApifyClient('apify_api_czHu0VeyZoxP9fEApsHUDLJwmKzf0e01ymiK')
# Prepare the Actor input

run_input = {
    "searchTerms": ["webscraping"],
    "isMultiple": False,
    "timeRange": "",
    "geo": "",
    "viewedFrom": "",
    "skipDebugScreen": False,
    "isPublic": False,
    "category": "",
    "maxItems": 0,
    "maxConcurrency": 10,
    "maxRequestRetries": 7,
    "pageLoadTimeoutSecs": 180,
}


# Run the Actor and wait for it to finish

run = client.actor("DyNQEYDj9awfGQf9A").call(run_input=run_input)


# Fetch and print Actor results from the run's dataset (if there are any)

for item in client.dataset(run["defaultDatasetId"]).iterate_items():

    print(item)